In [1]:
from tkinter import *
import tkinter.messagebox
import math
import time

In [2]:
#Cell class to store a state's 
#possible movements, distance to another point
class Cell:  
    def __init__(self,x,y):
        self.x = x
        self.y = y
    def right(self):
        return Cell(self.x,self.y+1)
    def left(self):
        return Cell(self.x,self.y-1)
    def up(self):
        return Cell(self.x-1,self.y)
    def down(self):
        return Cell(self.x+1,self.y)
    #Euclidean Distance of 2 cells
    def distance(self,other):
        return math.sqrt(((other.x-self.x)**2)+((other.y-self.y)**2))
    def isEqual(self,other):
        return self.x==other.x and self.y==other.y
    
class Model:
    
    def __init__(self,grid_width,grid_height):
        #grid_width and grid_height specify the simulation environement 
        self.grid_width = grid_width
        self.grid_height = grid_height
        self.empty = " " # Empty cell
        self.ped = "P" # Pedestrian
        self.obs = "O" # Obstacle
        self.tar = "T" # Target
        #obstacles in the simulation
        self.obstacles = []
        #pedestrians in the simulation
        self.pedestrians = [Cell(0,0)]
        # Boolean array to check 
        # if a pedestrian reached the target 
        self.reachedPedestrians = [False for i in range(len(self.pedestrians))]
        #targets in the simulation
        self.targets = [Cell(3,2)]
        
        self.grid = self.createGrid()
        self.placeStates()
        
    #Creates initial simulation with empty cells
    def createGrid(self):
        return [ [self.empty]*self.grid_width for i in range(self.grid_height)]
    
    #Places pedestrians,obstacles and targets in the grid
    def placeStates(self):
        for p in self.pedestrians:
            self.grid[p.x][p.y] = self.ped
        for p in self.obstacles:
            self.grid[p.x][p.y] = self.obs
        for p in self.targets:
            self.grid[p.x][p.y] = self.tar
            
    #Method checks if all pedestrians reach a target.
    def allReached(self):
        for p in self.reachedPedestrians:
            if not p:
                return False
        return True
    
    #Updates grid based on the movement of current pedestrian
    def update(self,p,shortestCell,i,reached):
        self.grid[p.x][p.y] = self.empty
        self.grid[shortestCell.x][shortestCell.y] = self.ped
        self.pedestrians[i].x = shortestCell.x
        self.pedestrians[i].y = shortestCell.y
        if reached:
            self.reachedPedestrians[i] = True

    #Simulates each pedestrian one step 
    # according to shortest distance move
    def simulateOneStep(self):
        for i in range(len(self.pedestrians)):
            if not self.reachedPedestrians[i]:
                p = self.pedestrians[i]
                shortestCell,reached = self.findShortestMove(p)
                self.update(p,shortestCell,i,reached)
    #Checks if a cell is in the grid and movable
    def isValidCell(self,p):
        return p.x>=0 and p.x<self.grid_height and p.y>=0 and p.y<self.grid_width and self.grid[p.x][p.y] == self.empty
    
    # Finds shortest distance 1 step move
    # returns cell with shortest distance to the current pedestrian
    # returns reached: True if the current pedestrian reached the target
    def findShortestMove(self,p):
        currentDistance =p.distance(self.targets[0])
        shortestCell = p
        for target in self.targets:
            if self.isValidCell(p.left()) and p.left().distance(target) < currentDistance:
                currentDistance = p.left().distance(target)
                shortestCell = p.left()
            if self.isValidCell(p.right()) and p.right().distance(target) < currentDistance:
                currentDistance = p.right().distance(target)
                shortestCell = p.right()
            if self.isValidCell(p.up()) and p.up().distance(target) < currentDistance:
                currentDistance = p.up().distance(target)
                shortestCell = p.up()
            if self.isValidCell(p.down()) and p.down().distance(target) < currentDistance:
                currentDistance = p.down().distance(target)
                shortestCell = p.down()
        # If pedestrian reaches target distance between
        # target and pedestrian should be 1
        if currentDistance == 1:
            reached = True
        else:
            reached = False
        return shortestCell,reached
    



In [3]:
#Method to display simulation grid
def display(simulation):
    boxSize=16
    for i in range(simulation.grid_height):
        for j in range(simulation.grid_width):
            if simulation.grid[i][j] == simulation.ped:
                canvas.create_rectangle(50+boxSize*j, 25+boxSize*i, 50+boxSize*(j+1), 25+boxSize*(i+1), fill="red",outline="black")
                canvas.create_text(50+boxSize*j+boxSize/2,25+boxSize*i+boxSize/2,text =simulation.ped)
            elif simulation.grid[i][j] == simulation.obs:
                canvas.create_rectangle(50+boxSize*j, 25+boxSize*i, 50+boxSize*(j+1), 25+boxSize*(i+1), fill="purple",outline="black")
                canvas.create_text(50+boxSize*j+boxSize/2,25+boxSize*i+boxSize/2,text =simulation.obs)
            elif simulation.grid[i][j] == simulation.tar:
                canvas.create_rectangle(50+boxSize*j, 25+boxSize*i, 50+boxSize*(j+1), 25+boxSize*(i+1), fill="goldenrod",outline="black")
                canvas.create_text(50+boxSize*j+boxSize/2,25+boxSize*i+boxSize/2,text =simulation.tar)
            else:
                canvas.create_rectangle(50+boxSize*j, 25+boxSize*i, 50+boxSize*(j+1), 25+boxSize*(i+1), fill="white",outline="black")
                canvas.create_text(50+boxSize*j+boxSize/2,25+boxSize*i+boxSize/2,text =simulation.empty)

In [4]:

grid_width = 5
grid_height = 5
simulation = Model(grid_width,grid_height)
root = tkinter.Tk()
root.title("Cellular Automaton")
root.resizable(width=False,height=False)
root.geometry("1000x1000")
canvas = tkinter.Canvas(root, width=550, height=550)
currentGrid = simulation.grid
display(simulation)

canvas.grid(row = 0, column = 10, sticky = E, pady = 2)
#Simulates the grid until either each pedestrian reaches the target
#or gets stuck in a constant position
#Displays the final states of simulation
def finalStep():
    while not simulation.allReached():
        nextStep()
#Method to simulate one step on GUI
def nextStep():
    canvas.delete("all")
    simulation.simulateOneStep()
    currentGrid = simulation.grid
    display(simulation)
    
#Removes a pedestrian, obstacle or target from the position received by user input
def emptyCell():
    cellToDelete = Cell(rowNum.get(),colNum.get())
    for i in range(len(simulation.pedestrians)):
        if simulation.pedestrians[i].isEqual(cellToDelete):
            del simulation.pedestrians[i]
            del simulation.reachedPedestrians[i]
            break
    for i in range(len(simulation.obstacles)):
        if simulation.obstacles[i].isEqual(cellToDelete):
            del simulation.obstacles[i]
            break
    for i in range(len(simulation.targets)):
        if simulation.targets[i].isEqual(cellToDelete):
            del simulation.targets[i]
            break
    simulation.grid[cellToDelete.x][cellToDelete.y] = simulation.empty
    display(simulation)
#Adds new pedestrian to the position received by user input
def addPedestrian():
    cellToAdd = Cell(rowNum.get(),colNum.get())
    simulation.pedestrians.append(cellToAdd)
    simulation.grid[cellToAdd.x][cellToAdd.y] = simulation.ped
    simulation.reachedPedestrians.append(False)
    display(simulation)
#Adds new obstacle to the position received by user input
def addObstacle():
    cellToAdd = Cell(rowNum.get(),colNum.get())
    simulation.obstacles.append(cellToAdd)
    simulation.grid[cellToAdd.x][cellToAdd.y] = simulation.obs
    display(simulation)
#Adds new target to the position received by user input
def addTarget():
    cellToAdd = Cell(rowNum.get(),colNum.get())
    simulation.targets.append(cellToAdd)
    simulation.grid[cellToAdd.x][cellToAdd.y] = simulation.tar
    display(simulation)
    
#Buttons for GUI
clickButton = Button(root, text = "NEXT STEP", command = nextStep).grid(row = 0, column = 30)
clickButton2 = Button(root, text = "FINAL STEP", command = finalStep).grid(row = 1, column = 30)
rowNum = tkinter.IntVar()
colNum = tkinter.IntVar()
rowLabel = Label(root, text = 'Row Number [0-9]').grid(row = 3, column = 30)
rowEntry = Entry(root,textvariable = rowNum).grid(row = 4, column = 30)
colLabel = Label(root, text = 'Column Number [0-9]').grid(row = 5, column = 30)
colEntry=Entry(root, textvariable = colNum).grid(row = 6, column = 30)
addPedestrian = Button(root, text = "Add Pedestrian", command = addPedestrian).grid(row = 7, column = 30)
addObstacle = Button(root, text = "Add Obstacle", command = addObstacle).grid(row = 8, column = 30)
addTarget = Button(root, text = "Add Target", command = addTarget).grid(row = 9, column = 30)
emptyCell = Button(root, text = "Delete Cell", command = emptyCell).grid(row = 10, column = 30)
root.mainloop()